# Criando rede de coautores
Esse notebook contém o código para criar a rede de coautores a partir da base de dados dos currículos dos
pesquisadores da área de Matemática.

Na rede, cada pesquisador é um nó e as arestas representam o fato de dois pesquisadores já terem publicados
pelo menos um artigo em conjunto. A espessura da aresta representa a quantidade de publicações conjuntas.
Note que a rede é não direcional, não importando a ordem de autoria.
Para a construção da rede, consideraram-se somente os artigos publicados em periódicos.

A metodologia implementada para a criação da rede consiste em iterar por cada pesquisador, comparando-o com
os demais de forma a encontrar um trabalho em comum. Para identificar se há artigo escrito em conjunto,
os nomes dos coautores do pesquisador será comparado com o campo "NOME-EM-CITACOES-BIBLIOGRAFICAS"
do outro pesquisador. Caso algum dos nomes seja igual, será estabelecida uma conexão.

É possível que hajam erros nos nomes dos coautores apontados por cada pesquisador, assim, para
buscar reduzir erros por conta de erros de escrita, iremos em seguida buscar pelo nome dos artigos
publicados e ano de publicação. Iremos introduzir um "flag" nesses casos, para verificar manualmente se de fato a
conexão estabelecida entre os pesquisadores está correta. Caso apareçam muitos falsos positivos,
iremos buscar uma outra chave mais restrita, como adicionar o nome da revista em que foi publicado.

Uma vez definidos os pesquisadores conectados, iremos realizar a contagem dos trabalhos conjuntos,
para assim estabelecer a espessura das arestas da rede.

Formato para a base de geração dos grafos:
Lista de Arestas
Pesquisador A | Pesquisador B | Qtd de Trabalhos Juntos


*Foram considerados os currículos indexados na plataforma do Currículo Lattes com área igual a "Matemática",
"Probabilidade e Estatística" ou "Ciência da Computação".*

In [113]:
import zipfile
import json
import xmltodict
import os
from collections import Counter

# Encontrando arquivos com erro na coleta
O motivo do erro é que o id indicado por algum motivo não está mais atrelado a um currículo.

In [330]:
zipfs = ['./dados/' + zipf for zipf in os.listdir('./dados/')][0:100]
zips_erros = []
for i, zipcv in enumerate(zipfs):
    cv = xmltodict.parse((zipfile.ZipFile(zipcv,'r').read('curriculo.xml')))
    try:
        print(i,cv['ERRO'])
        zips_erros.append(i)
    except:
        
        pass

In [331]:
zips_erros

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [306]:
zipfs = ['./dados/' + zipf for zipf in os.listdir('./dados/')][0:20]
edges = []
for i, zipcv1 in enumerate(zipfs):
    print(zipcv1)
    cv1 = xmltodict.parse((zipfile.ZipFile(zipcv1,'r').read('curriculo.xml')))
    coautores = Get_Coautores(cv1)
    for zipcv2 in zipfs[i+1:]:
        print(zipcv2)
        cv2 = xmltodict.parse((zipfile.ZipFile(zipcv2,'r').read('curriculo.xml')))
        link= Check_Link(cv1,cv2,coautores_cv1=coautores)
        if (link!=['']):
            edges.append(link)

./dados/6115398975075240.zip
./dados/1386464075867571.zip
./dados/9312332483222873.zip
./dados/4070657423906906.zip
./dados/1807540836275750.zip
./dados/9670752954194989.zip
./dados/1855466912397758.zip
./dados/5603369249661646.zip
./dados/2989485146353807.zip
./dados/1152911680785585.zip
./dados/4860258252593975.zip
./dados/6877649058117092.zip
./dados/8459439898470769.zip
./dados/3638688119433520.zip
./dados/324248447909181.zip


KeyError: 'CURRICULO-VITAE'

In [299]:
edges

[]

In [290]:
def Get_Coautores(cv):
    coautores = []
    if list == type(cv['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']):
        for artigo in cv['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']: 
            if list == type(artigo['AUTORES']):
                for i,autor in enumerate(artigo['AUTORES']):
                    coautor = autor['@NOME-PARA-CITACAO'].lower()
                    coautores.append(coautor)
            else:
                coautor = artigo['AUTORES']['@NOME-PARA-CITACAO'].lower()
                coautores.append(coautor)
        coautores = dict(Counter(coautores))
    else:
        artigo  = cv['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']
        if list == type(artigo['AUTORES']):
            for i,autor in enumerate(artigo['AUTORES']):
                coautor = autor['@NOME-PARA-CITACAO'].lower()
                coautores.append(coautor)
        else:
            coautor = artigo['AUTORES']['@NOME-PARA-CITACAO'].lower()
            coautores.append(coautor)
    coautores = dict(Counter(coautores))
        
    return coautores

def Check_Link(cv1,cv2,coautores_cv1):
    autor_check = cv2['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-EM-CITACOES-BIBLIOGRAFICAS'].lower().split(';')
    if bool(set(coautores.keys()).intersection(autor_check)):
        return [cv1['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR'],
                cv2['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']]
    else:
        return ['']
    

In [307]:
cvzip   =  './dados/324248447909181.zip' # Elon Lages Lima

cv = xmltodict.parse((zipfile.ZipFile(cvzip,'r').read('curriculo.xml')))


In [311]:
cv['ERRO']

OrderedDict([('MENSAGEM', 'Erro ao recuperar o XML')])

In [308]:
Get_Coautores(cv)

KeyError: 'CURRICULO-VITAE'

In [288]:
cv['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']#['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']

OrderedDict([('TRABALHOS-EM-EVENTOS',
              OrderedDict([('TRABALHO-EM-EVENTOS',
                            OrderedDict([('@SEQUENCIA-PRODUCAO', '28'),
                                         ('DADOS-BASICOS-DO-TRABALHO',
                                          OrderedDict([('@NATUREZA',
                                                        'COMPLETO'),
                                                       ('@TITULO-DO-TRABALHO',
                                                        'An initial study on high-dimensional data visualization through subspace clustering'),
                                                       ('@ANO-DO-TRABALHO',
                                                        '2015'),
                                                       ('@PAIS-DO-EVENTO',
                                                        'Brasil'),
                                                       ('@IDIOMA', 'Inglês'),
                                              

In [ ]:
#